<a href="https://colab.research.google.com/github/cvs2010/udemyDeepLearning/blob/main/scraping_data_zapimoveis_df.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importando as bibliotecas
import bs4 
from bs4 import BeautifulSoup
import urllib.request as urllib_request
from urllib.request import Request, urlopen, urlretrieve
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
import pandas as pd
import re

print("BeautifulSoup ->", bs4.__version__)
print("urllib ->", urllib_request.__version__)
print("pandas ->", pd.__version__)

BeautifulSoup -> 4.11.2
urllib -> 3.10
pandas -> 1.5.3


In [8]:
#Declarando variável cards
cards = []
garagem = []
quartos = []
rua = []
bairro = []

##Obtendo HTML e o total de paginas
pages = int(2)


##interando por todas as paginas do site
for i in range(pages):
    ##Obtendo o HTML
   
    response = 'https://www.zapimoveis.com.br/venda/casas/?pagina=' + str(i) + '&transacao=Venda&tipoUnidade=Residencial,Casa&tipo=Im%C3%B3vel%20usado'
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

    requisicao = Request(response,headers = headers)

    try:
        response = urlopen(requisicao)
        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')
    except HTTPError as e:
        print(e.status(), e.reason())
    except URLError as e:
        print(e.reason()) 

    
    #Obtendo as Tags de interesse
    anuncios = soup.find('div',{'class':'listings__container'}).findAll('div',class_='card-listing simple-card')

    #Coletando as informações dos CARDS
    for anuncio in anuncios:
        card = {}

        # Valor
        valor = anuncio.find('p', {'class': 'simple-card__price js-price color-darker heading-regular heading-regular__bolder align-left'})
        if valor == None:
            card['valor'] = int('0')
        else :
            card['valor'] = int(valor.get_text().replace('R$','').replace('\n','').replace('.',''))
            
        
        #descricao
        descricao = anuncio.find('span',{'class':'simple-card__text text-regular'})
        if descricao == None:
            card['descricao'] = str('Vazio')
        else :
            card['descricao'] = str(descricao.get_text().replace('-',' ').replace('\n',''))


        #endereco
        endereco = anuncio.find('h2',{'class':'simple-card__address color-dark text-regular'})
        if endereco == None:
            card['local'], card['bairro/cidade'] = str('Não disponivel', 'Não disponivel')
        else :
            card['local'], card['bairro/cidade'], *outro = endereco.get_text().replace('\n','').replace('  ','').split(',')

            
        #quartos
        quartos = anuncio.find('span',{'itemprop':'numberOfRooms'})
        if quartos == None:
            card['quartos'] = int(1)
        else :
            card['quartos'] = int(quartos.get_text().replace(' ','').replace('\n',''))
        

        
        #vagas
        garagem = anuncio.find('li', {'class': 'feature__item text-small js-parking-spaces'})
        if garagem is None:
            card['garagem'] = 0
        else:
            text = garagem.get_text().replace(' ', '').replace('\n', '')
            digits = re.findall(r'\d+', text)
            if digits:
                card['garagem'] = int(digits[0])
            else:
                card['garagem'] = 0

        
            
        #banheiros
        banheiro = anuncio.find('span',{'itemprop':'numberOfBathroomsTotal'})
        if banheiro == None:
            card['banheiro'] = int(1)
        else :
            card['banheiro'] = int(banheiro.get_text().replace(' ','').replace('\n',''))

          
        #metragem
        metro = anuncio.find('span',{'itemprop':'floorSize'})
        if metro == None:
            card['area(m2)'] = int(30)
        else :
            card['area(m2)'] = int(metro.get_text().replace(' ','').replace('\n','').replace('m²',''))


            
        #IPTU
        iptu = anuncio.find('li',{'class':'card-price__item iptu text-regular'})
        if iptu == None:
            card['iptu'] = int('0')
        else :
            card['iptu'] = int(iptu.get_text().replace('R$','').replace('\n','').replace('.','').replace('IPTU','').replace(' ',''))

                          
            
        #condominio
        condominio = anuncio.find('li', {'class': 'card-price__item condominium text-regular'})
        if condominio is None:
            card['condominio'] = 0
        else:
            text = condominio.get_text().replace('condomínioR$', '').replace(' ', '').replace('.', '')
            digits = re.findall(r'\d+', text)
            if digits:
                card['condominio'] = int(digits[0])
            else:
                card['condominio'] = 0
        
        
        #tipo
        tipo = anuncio.find('small')      
        if tipo == None:
            card['novidade'] = str('Nenhuma')
        else :
            card['novidade'] = str(tipo.get_text())
        
        #suite    
        if re.search("suite|suíte|suites|suítes", card['descricao'].lower()) != None:
            card['suite'] =  int(1)
        else:
            card['suite'] =  int(0)  
        
        
        #acabamento
        if re.search("porcelanato|ceramica|cerâmica|cerâmico|cerâmicos|gesso|antiderrapante|planejado", card['descricao'].lower()) != None:
            card['acabamento'] =  int(1)
        else:
            card['acabamento'] =  int(0)  
         
        cards.append(card)


        
dataset = pd.DataFrame(cards)
print(dataset.isna().sum())
dataset.to_csv('./dataset-casas-geral.csv', sep=';' , index=False, encoding='utf-8-sig')
dataset

valor            0
descricao        0
local            0
bairro/cidade    0
quartos          0
garagem          0
banheiro         0
area(m2)         0
iptu             0
condominio       0
novidade         0
suite            0
acabamento       0
dtype: int64


,valor,descricao,local,bairro/cidade,quartos,garagem,banheiro,area(m2),iptu,condominio,novidade,suite,acabamento
0,230000,Entre em contato Garcia Corretor Casa 2 quar...,Rua BV 25,Residencial Boa Vista II,2,2,1,76,850,0,Super destaque,1,0
1,0,CASA ASSOBRADADA PISO TERREO GARAGEM PARA ATE ...,Parque Residencial Villa dos Inglezes,Sorocaba,3,4,5,225,1400,310,Super destaque,1,1
2,915000,Ótima casa com terreno de 11m x 40m onde se en...,Rua Doutor Barcelos,Tristeza,3,4,2,250,260,0,Super destaque,0,0
3,300000,A casa no bairro Parque Pinheiros com 130 metr...,Rua Rosa Provencial Delgado,Parque Pinheiros,2,2,1,130,1000,0,Nenhuma,1,0
4,300000,A casa no bairro Vila Jaguara tem 130 metros q...,Rua Paúva,Vila Jaguara,2,1,2,130,1000,0,Nenhuma,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,Casa bem localizada com 2 quartos sendo 1 suít...,Rua Bueno de Paiva,Independência,2,2,2,122,1,0,Nenhuma,1,0
196,425000,Residência térrea muito bem localizada no Jard...,Rua Mário Ferreira de Souza,Jardim Eugenio Mendes Lopes,3,2,3,150,83,0,Super destaque,1,0
197,290000,Casa à venda Bairro São Franscisco Igarapé E...,Marechal Rondon,Igarapé,2,0,2,360,0,0,Nenhuma,1,0
198,125000,A casa no bairro Carananduba (Mosqueiro) com 9...,Estrada do Carananduba,Carananduba Mosqueiro,4,3,2,95,0,0,Nenhuma,1,0


In [ ]:
#Declarando variável cards
cards = []
garagem = []
quartos = []
rua = []
bairro = []

##Obtendo HTML e o total de paginas
pages = int(350)


##interando por todas as paginas do site
for i in range(pages):
    ##Obtendo o HTML
   
    response = 'https://www.zapimoveis.com.br/venda/apartamentos/?pagina='+str(i)
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

    requisicao = Request(response,headers = headers)

    try:
        response = urlopen(requisicao)
        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')
    except HTTPError as e:
        print(e.status(), e.reason())
    except URLError as e:
        print(e.reason()) 

    
    #Obtendo as Tags de interesse
    anuncios = soup.find('div',{'class':'listings__container'}).findAll('div',class_='card-listing simple-card')

    #Coletando as informações dos CARDS
    for anuncio in anuncios:
        card = {}

        # Valor
        valor = anuncio.find('p', {'class': 'simple-card__price js-price color-darker heading-regular heading-regular__bolder align-left'})
        if valor == None:
            card['valor'] = int('0')
        else :
            card['valor'] = int(valor.get_text().replace('R$','').replace('\n','').replace('.',''))
            
        
        #descricao
        descricao = anuncio.find('span',{'class':'simple-card__text text-regular'})
        if descricao == None:
            card['descricao'] = str('Vazio')
        else :
            card['descricao'] = str(descricao.get_text().replace('-',' ').replace('\n',''))


        #endereco
        endereco = anuncio.find('h2',{'class':'simple-card__address color-dark text-regular'})
        if endereco == None:
            card['local'], card['bairro/cidade'] = 'Não disponivel', 'Não disponivel'
        else :
            card['local'], card['bairro/cidade'], *outro = endereco.get_text().replace('\n','').replace('  ','').split(',')

            
        #quartos
        quartos = anuncio.find('span',{'itemprop':'numberOfRooms'})
        if quartos == None:
            card['quartos'] = int(1)
        else :
            card['quartos'] = int(quartos.get_text().replace(' ','').replace('\n',''))
        

        
        #vagas
        garagem = anuncio.find('li',{'class':'feature__item text-small js-parking-spaces'})
        if garagem == None:
            card['garagem'] = int(0)
        else :
            card['garagem'] = int(garagem.get_text().replace(' ','').replace('\n',''))


        #banheiros
        banheiro = anuncio.find('span',{'itemprop':'numberOfBathroomsTotal'})
        if banheiro == None:
            card['banheiro'] = int(1)
        else :
            card['banheiro'] = int(banheiro.get_text().replace(' ','').replace('\n',''))

            
        #metragem
        metro = anuncio.find('span',{'itemprop':'floorSize'})
        if metro == None:
            card['area(m2)'] = int(30)
        else :
            card['area(m2)'] = int(metro.get_text().replace(' ','').replace('\n','').replace('m²',''))


        #IPTU
        iptu = anuncio.find('li',{'class':'card-price__item iptu text-regular'})
        if iptu == None:
            card['iptu'] = int('0')
        else :
            card['iptu'] = int(iptu.get_text().replace('R$','').replace('\n','').replace('.','').replace('IPTU','').replace(' ',''))


        #condominio
        condominio = anuncio.find('li',{'class':'card-price__item condominium text-regular'})
        if condominio == None:
            card['condominio'] = int('0')
        else :
            card['condominio'] = int(condominio.get_text().replace('condomínioR$','').replace(' ','').replace('.',''))
        
        #tipo
        tipo = anuncio.find('small')      
        if tipo == None:
            card['novidade'] = str('Nenhuma')
        else :
            card['novidade'] = str(tipo.get_text())
            
         #suite    
        if re.search("suite|suíte|suites|suítes", card['descricao'].lower()) != None:
            card['suite'] =  int(1)
        else:
            card['suite'] =  int(0)
        
        #academia
        if re.search("academia|academias", card['descricao'].lower()) != None:
            card['academia'] =  int(1)
        else:
            card['academia'] =  int(0)
        
        #varanda
        if re.search("varanda|terraço|terraco|sacada|varandas|terraços|terracos|sacadas", card['descricao'].lower()) != None:
            card['varanda'] =  int(1)
        else:
            card['varanda'] =  int(0)
        
        #transporte
        if re.search("estação|estaçao|metro|onibus|ônibus", card['descricao'].lower()) != None:
            card['transporte'] =  int(1)
        else:
            card['transporte'] =  int(0)
        
        #salao
        if re.search("salao|salão", card['descricao'].lower()) != None:
            card['salao'] =  int(1)
        else:
            card['salao'] =  int(0)
        
        #planejado
        if re.search("planejado|planejados|planejada|planejadas", card['descricao'].lower()) != None:
            card['planejado'] =  int(1)
        else:
            card['planejado'] =  int(0)    
                        
        cards.append(card)

dataset = pd.DataFrame(cards)
print(dataset.isna().sum())
dataset.to_csv('./dados/dataset-ap-geral.csv', sep=';', index=False, encoding='utf-8-sig')
dataset